# <span style="color:blue"> *Version 0* </span>

# Lab#3 - Exploring the LTE Architecture

This lab intends to explore the LTE architecture and the attachment procedure for a user equipment (UE). The lab starts with an introduction of the LTE architecture followed by the attachment process itself encompassing different connection stages as UE initiates its connection with the LTE network. Afterwards, a simulation encompassing different connection establishment and attachment processes is implemented using ns-3 LTE simulator. The students will be encouraged to design their own LTE radio access network (RAN) topologies using React app discussed in detail in Lab0. A summary of the lab objectives is provided as below:

- Understand the RAN, EPC, and, interconnections between these LTE sub-networks
- Demonstration the UE attachment process
- Understanding the user and control plane in LTE

The scope of this lab covers topics from <b>Sections 4.1</b> (pp. 203),<b>4.2</b> (pp. 206-218), <b>4.6.1</b> (pp. 247)</b>, and <b>4.6.2</b> (pp. 250) of textbook <a href="#References">[1] </a>.</p>

## 1. Introduction

### 1.1 LTE Architecture 
The LTE network is divided into two parts: a evolved universal mobile telecommunications system terrestrial radio access network (E-UTRAN) and an evolved packet core network as shown in <a id='fig1'>[Figure. 1](#fig1)</a> . These subnetworks separate the user access and network core with different nodes for varying roles in LTE infrastructure.

| |
|---|
| ![LTE_net.png](Figures/LTE_net.png) |

<a id='fig1'><p style="text-align: center;"><b>Figure. 1: LTE Architecture.</b></p></a>

<p style='text-align: justify;'> Some visible differences in the architecture from Universal Mobile Telecommunications System (UMTS) UTRAN are the introduction of the X2 interface and the no Radio Network Controller in the E-UTRAN. The S1 interface is the connection between the RAN and EPC utilized for the LTE services such as traditional calling, voice over IP (VoIP) and dedicated data access towards the internet. It has two different interface types based on utilization by the user plane and control plane protocols. S1-U is used for user plane and S1-AP for the application or control plane. In addition any signaling between RAN and EPC is also carried via S1. EPC has its own interfaces for performing different management functions related to the LTE processes such as mobility and location management or user information management.<br>
</p>
    

### 1.2 LTE Protocol Stack 

The LTE protocol stack as depicted in <a id='fig2'>[Figure. 2](#fig2)</a>, which consists of the following protocols for different functionalities:


| |
|---|
| ![lte-stack.png](Figures/lte-stack.png) |
<a id='fig2'><p style="text-align: center;"><b>Figure. 2: LTE Protocol Stack. (Reproduced from <a href="#References">[2])</b></a>    
    
<p style='text-align: justify;'>
- <b>Radio Resource Control (RRC)</b>: The broadcast of system information related to the non-access stratum (NAS) and broadcast of system information related to the access stratum (AS) are the key services and functions of the RRC sublayer. In addition it is also responsible for paging, configuration, maintenance, and release of point-to-point radio bearers, as well as the establishment, maintenance, and release of an RRC connection between the UE and E-UTRAN <a href="#References">[3] </a>.</p>
<p style='text-align: justify;'>
- <b>Packet Data Convergence Protocol (PDCP)</b>: For radio bearers mapped on RLC acknowledged mode (AM), the PDCP layer is in charge of header compression and decompression of IP data, transfer of data (user plane or control plane), maintenance of PDCP Sequence Numbers (SNs), in-sequence delivery of upper layer packet data units(PDUs) at re-establishment of lower layers, duplicate elimination of lower layer service data units (SDUs) at re-establishment of lower layers, ciphering and deciphering of UE data <a href="#References">[3] </a>.</p>
<p style='text-align: justify;'>
- <b>Radio Link Control (RLC)</b>: RLC runs in three different modes: acknowledged mode (AM), unacknowledged mode (UM), and transparent mode (TM). The transfer of upper layer PDUs, error correction via ARQ (only for AM data transfer), concatenation, segmentation, and reassembly of RLC SDUs are all handled by the RLC Layer (Only for UM and AM data transfer). Additionally, the RLC is in charge of re-ordering RLC data PDUs (only for UM and AM data transfer), re-segmenting RLC data PDUs (only for AM data transfer), detecting duplicates (only for UM and AM data transfer), discarding RLC SDUs (only for UM and AM data transfer), re-establishing the RLC, and detecting protocol errors (Only for AM data transfer) <a href="#References">[3] </a>.</p>

### 1.3 LTE Connection Setup

As soon as a UE is turned on, a set of procedures is adopted to ensure that the communication channel between eNB and UE is able to support the required services upon request. The connection setup is important to consider since this process initiates all the other processes in LTE including attachment and handovers. Certain information exchange must be completed during initial stages of this process. This is know as the system information (SI), which is transmitted by LTE eNB on DL channel Broadcast Control Channel (BCCH). The SI is divided into two parts: the static part and the dynamic part.

- <p style='text-align: justify;'><b>Master Information Blocks (MIB)</b>, also known as static parts, are transferred once every 40 microseconds.</p>
- <p style='text-align: justify;'><b>System Information Blocks (SIB)</b>, also known as the dynamic parts, are composed of various message kinds that are sent every 80, 160, 320, … milliseconds.</p>

<p style='text-align: justify;'>In this lab, we will only be concerned with the MIB, SIB1, and SIB2 that carry the following information:</p>

- <p style='text-align: justify;'><b>MIB</b>: The channel bandwidth, transmit power, number of antennas, SIB scheduling, and other relevant data required for radio access.</p>
- <p style='text-align: justify;'><b>SIB1</b>: Information about whether or not the UE is permitted to access the LTE cell. It also determines how the other SIBs are scheduled. Cell ID, mobile country code (MCC), mobile network code (MNC), tracking area code (TAC), and SIB mapping are all included.</p>
- <p style='text-align: justify;'><b>SIB2</b>: Both common and shared channel information. RRC, uplink power control, preamble power ramping, uplink cyclic prefix (CP) length, sub-frame hopping, and uplink E-UTRA Absolute Radio Frequency Channel Number (EARFCN).</p>

<b>Note:</b> The textbook reference for these information blocks is <b>Section 4.3.8</b> (pp. 229) <a href="#References">[1] </a>.

### 1.4 UE eNB and RRC States

The state transitions for UE and eNB will signify the reception of these information blocks as well as the current state of these nodes. These are dictated by the following state diagrams in <a id='fig3'>[Figure. 3](#fig3)</a>. 

| |
|---|
| ![lab3-rrc-states.png](Figures/lab3-rrc-states.png) |
<a id='fig3'><p style="text-align: center;"><b>[Figure. 3: LTE RRC State Diagram, (a) UE, (b) eNB](#fig3). (Source <a href="#References">[4] )</b></a>   

#### 1.4.1 Connection setup procedure
The connection setup starts with the initiation of the <u>cell search</u> process initiated by a UE that has just been turned on. It searches for a suitable cell to attach by analyzing the received signal strength from different prospective cells in the area. During this process, the UE waits for the reception of MIB and SIB messages in order to collect required information about the connection. Upon reception of SIB1 and SIB2, the <u>random access</u> (RA) procedure is initiated in order to exchange more information and allow the eNB and EPC to reserve resources for upper layer communications (RRC, PDCP). The success of RA leads to the initiation of RRC connection setup with the eNB and eventual connection of eNB with EPC via S1-AP protocol. At this point, the UE is connected to the LTE network and ready to utilize the infrastructure.

This process is implemented as a finite state machine as depicted in <a id='fig3'>[Figure. 3](#fig3)</a>.

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
## Design and Simulating an LTE Network in ns3

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## <span style="color:red"> *Task#1: Radio Access Network, Enhanced Packet Core and Interactions* </span>
This task involves the creation of a basic one cell RAN topology using the React application followed by the simulation and analysis of connection setup traces using ns-3. Hence, the overall learning objectives are presented as the following deliverables:

### 2.1 Designing a RAN topology
The topology design follows the same basic steps from lab0 starting from accessing the react app, design and saving of the topology . Kindly utilize the blueprint provided in <a id='fig4'>[Figure. 4](#fig4)</a> to design the topology where the actual location of eNB and UEs is not important as long as the users are inside the coverage area of eNB. The <b>eNB TXPower = 45dBm</b> for this task.

| |
|---|
| ![lab3-task1-topology.png](Figures/lab3-task1-topology.png) |
<a id='fig4'><p style="text-align: center;"><b>[Figure. 4: LTE RAN topology](#fig4).</b></a>

> <b>Note:</b> Access the React application via: http://vm-public-ip:3000/.

### 2.2 Simulate the designed topology in ns-3
The python code in `lab3-init.py` is provided to convert the design topology in a form acceptable to the SEM and ns-3 simulation engines. Moreover, some relevant command line arguments are also passed to the ns-3 program implementing the given topology in order to observe the results and status of the simulation. 

> <b>Note:</b> The python code will display the saved topologies from the react app and we can choose any one of them to simulate.

The ns-3 simulator expects the values provided in the form of UE and eNB location from the react app in order to design the topology. Several other variables are also provided in the `lab3-init.py` script in order to provide sufficient information for the creation and operation of simulation. This includes the configuration related to the center frequency, bandwidth, attachment preferences, additional noise modeling. The noise model utilized is a simple randomized threshold added in the UE and eNB. This contributes to the already observed interference noise experienced in the channel propagation. The ns-3 program also establishes several UDP streams in order to establish data communication and these are utilized for measurements of different performance metric. After successful completion of the simulation, the measured performance metrics are passed on back to the python script by the ns-3 program for data processing.

In [ ]:
%run -i 'Modules/lab3-init.py'

### 2.3 Processing the simulation data
The python code in `lab3-DataPre.py` is provided to process and clean the obtained data from ns-3 simulations.

> <b>Note:</b> Two different levels of logs are collected to exhibit the state machines for UE and eNB as they experience different stages of connection establishment within RAN and with EPC as shown in <a id='fig3'>[Figure. 3](#fig3)</a>.
  

#### 2.3.1 Structure of Simulation Logs
The format of the observed logs is important to decode according to the contained information. <a id='fig5'>[Figure. 5](#fig5)</a> and <a id='fig6'>[Figure. 6](#fig6)</a> provides the description of a single observed log and its fields for RAN and EPC respectively:

| |
|---|
| ![lab3-logdesc.png](Figures/lab3-logdesc.png) |
<a id='fig5'><p style="text-align: center;"><b>[Figure. 5: RAN Log description](#fig5)</b></a>

| |
|---|
| ![lab3-epc-logs.png](Figures/lab3-epc-logs.png) |
<a id='fig6'><p style="text-align: center;"><b>[Figure. 6: EPC Log description](#fig6)</b></a>

In [ ]:
%run -i 'Modules/lab3-dataPre.py'

### <u>Exercises</u>

#### Q1. (a) Make sure that the simulation recognizes two UEs and one eNB for the designed topology. Which identifiers did you refer to?

<b> Answer:</b>

In [ ]:
## Note 1: use the output above, obtained from the simulation logs (refer to Figure 5 if needed)
## Note 2: if you don't find two UEs consider moving them closer to the eNB and repeating the simulation

# write down possible eNB identifiers
eNB_ids = ""


# write down possible UE identifiers
UE_ids = ""



 
#### Q1. (b) Identify any logs related to the establishment of LTE bearers?

<b> Answer:</b>

In [ ]:
## Note: Refer again to the same logs

bearers = ["Data Radio Bearer", "SRB1"]


#### Q2. What is a bearer in LTE? What is the difference between a dedicated and default bearer?

<b> Answer:</b>

#### Q3. Identify the random access procedure logs for one of the UEs from the available logs? All the involved messages must be identified.

> **Hint:** Preamble detection, random access, random access response are some of the messages


<b> Answer:</b>

#### Q4. What are the different functionalities of the MME and SGW in LTE networks?

<b> Answer:</b>

#### Q5. Describe the difference of the identification (IMSI, RNTI, etc.) of the UEs as observed ns-3 logs and in a real world LTE network like Telenor?

<b> Answer:</b>

---
---
### <span style="color:blue"> Milestone 1 </span>

Before proceeding, **call a TA** to make sure everything went as expected.

---
---

## <span style="color:red"> *Task#2: Attachment of UEs with the LTE Network* </span>
In this task, we will observe the connection setup (those implemented in ns-3) for the RAN and EPC parts of the network. The process starts with the communication between UEs and eNBs as the cell search begins with the attachment of users to eNBs. This is followed up by random access and the associated states of the UEs and eNBs upon reception of various communication messages from within RAN and the EPC. The state machine implementation in ns-3 for the eNB and UE RRC layers allows the users in the LTE RAN to transition through these states upon occurrence of various events. By observing the sequence of events, we can verify, optimize and potentially troubleshoot issues in the network.

### 3.1 Designing a RAN topology
Kindly utilize the blueprint provided in <a id='fig7'>[Figure. 7](#fig7)</a> to design the topology. The <b>eNB TXPower = 40dBm</b> for this task.

| |
|---|
| ![lab3-task2-topology.png](Figures/lab3-task2-topology.png) |
<a id='fig7'><p style="text-align: center;"><b>[Figure. 7: LTE RAN topology.](#fig7)</b></a>

> <b>Note:</b> Access the React application via: http://vm-public-ip:3000/.

### 3.2 Simulate the designed topology in ns-3
The python code in `lab3-init.py` is provided to convert the design topology in a form acceptable to the SEM and ns-3 simulation engines.

> <b>Note:</b> The python code will display the saved topologies from the react app and we can choose any one of them to simulate along with a value for <b>eNB power = 40dBm</b>.

In [ ]:
%run -i 'Modules/lab3-init.py'

### 3.3 Processing the simulation data
The python code in `lab3-dataPre.py` is provided to process and clean the obtained data from ns-3 simulations.

> <b>Note:</b> Two different levels of logs are collected to exhibit the state machines for UE and eNB as they experience different stages of connection establishment within RAN and with EPC as shown in <a id='fig3'>[Figure. 3](#fig3)</a>.
  

#### 3.3.1 Structure of Simulation Logs
The format of the observed logs is important to decode according to the contained information. <a id='fig5'>[Figure. 5](#fig5)</a> and <a id='fig6'>[Figure. 6](#fig6)</a> provides the description of a single observed log and its fields:

In [ ]:
%run -i 'Modules/lab3-dataPre.py'

### <u>Exercises</u>

#### Q1a. Identify the states of each UE until it is connected with the eNB (the completion of Attachment process).

<b> Answer:</b>

#### Q1b. List relevant logs from the simulation where necessary for Q1a.
> **Hint:** Check if all the users are present in the logs

<b> Answer:</b>

#### Q2a. Is there a problem with any of the UEs in the designed topology? Consult the state diagram in <a id='fig3'>[Figure. 3](#fig3)</a> and simulation logs to identify the problem.

> **Hint:** Analyze the logs to identify the state of each UE. This will help in identifying any problematic logs and the issue itself.

<b> Answer:</b>

#### Q2b. What can be done to resolve the problem identified in Q2a?

<b> Answer:</b>

#### Q3a. How is radio link failure issue detected and resolved in LTE?

<b> Answer:</b>

#### Q3b. Identify the relevant logs from simulation for Q3a?

<b> Answer:</b>

#### Q4. What is NAS and UserContext in LTE?

<b> Answer:</b>

---
---
### <span style="color:blue"> Milestone 2 </span>

Before proceeding, **call a TA** to make sure everything went as expected.

---
---

## <span style="color:red"> *Task#3: Boundary Conditions in Cellular Networks*</span>
In this task, students will simulate a cellular topology in order to find the cell boundary. The coordinates of the UE and eNB can be used to find the approximate coverage distance. This can be achieved by having a single UE and altering its location while keeping the **eNB txpower** parameter fixed. The boundary would be approximately beyond the point where UE is unable to communicate with the eNB.

> **Note:** Use the topology blueprint for designing a react topology as shown below in <a id='fig8'>[Figure. 8](#fig8)</a>.

| |
|---|
| ![lab3-task3-topology.png](Figures/lab3-task3-topology.png) |
<a id='fig8'><p style="text-align: center;"><b>[Figure. 8: LTE RAN topology](#fig8)</b></a>   


Add additional cells as required in the jupyter notebook and submit those additional cells as well during evaluation.
> Note: Run simulations for two positions for UE: 1) first while the connection is still normal with eNB, 2) second when the UE disconnects for the first time. The average of both these points will give us the approximate boundary points to be used for calculation boundary distance.

<b> Tx Power = 25dBm </b>

In [ ]:
%run -i 'Modules/lab3-init.py'

In [ ]:
%run -i 'Modules/lab3-dataPre.py'

<b> Tx Power = 35dBm </b>

In [ ]:
%run -i 'Modules/lab3-init.py'

In [ ]:
%run -i 'Modules/lab3-dataPre.py'

<b> Tx Power = 40dBm </b>

In [ ]:
%run -i 'Modules/lab3-init.py'

In [ ]:
%run -i 'Modules/lab3-dataPre.py'

### <u>Exercises</u>

#### Q1. What is the boundary of the cell for the following values of eNB txpower? If the mentioned condition for finding the boundary fails, specify for which value it happens?
- 25dBm
- 35dBm
- 40dBm

> Hint: Find the placement of the UE and eNB from the React app and calculate the distance between them to find the approximated cell coverage boundary. For a UE placed at (100, 100) and eNB at (0,0), distance can be calculated as follows:

$$\sqrt{(100-0)^2 + (100-0)^2} = 141.42$$

> Note: The positions of UE and eNB are selected from the designed topology when the connection is lost for the first time with the UE.

<b> Answer:</b>

#### Q2. Why are there different values for txpower for eNBs and UEs in the LTE standard?

<b> Answer:</b>

#### Q3. What are the main benefits of power control mechanism in LTE?

<b> Answer:</b>

---
---
### <span style="color:blue"> Milestone 3 </span>

At the end, **call a TA** to make sure everything went as expected.

---
---

# References

[1] "From GSM to LTE-Advanced Pro and 5G, An introduction to Mobile Networks and Mobile Broadband", Martin Sauter, 4th Edition, 2021

[2] " LTE Protocol Stacks", LTE Tutorials. Online: https://www.artizanetworks.com/resources/tutorials/pro_sta.html (Accessed: 11-01-2023)

[3] "LTE Quick Guide", LTE Tutorials Point. Online : https://www.tutorialspoint.com/lte/lte_quick_guide.htm (Accessed : 11-01-2023)

[4] "LTE Design Documentation", ns-3: LTE Module. Online: https://www.nsnam.org/docs/models/html/lte-design.html (Accessed : 11-01-2023)

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------